In [61]:
import sys
sys.path.append('../../')
from brokers.coinbase import Coinbase

cb = Coinbase()
symbol = 'BTC/USD'

# print(cb.exchange.fetch_my_trades(symbol))
# print(cb.exchange.fetch_balance())
# print(cb.exchange.create_market_buy_order(symbol, '0.001'))
print(cb.exchange.fetch_my_trades(symbol))
# print(cb.exchange.fetch_open_orders(symbol, limit = 5))
# print(cb.exchange.fetch_closed_orders(symbol, limit=5))

[{'id': '21715479', 'order': '2385316f-1ff2-4f12-a4fa-c9516b0848de', 'info': {'created_at': '2020-12-31T01:15:50.441Z', 'trade_id': 21715479, 'product_id': 'BTC-USD', 'order_id': '2385316f-1ff2-4f12-a4fa-c9516b0848de', 'user_id': '5fe53030706b8613d795bd7e', 'profile_id': 'b4df755c-aa96-4f36-ae26-42c742d3f465', 'liquidity': 'T', 'price': '28513.42000000', 'size': '0.01000000', 'fee': '1.4256710000000000', 'side': 'buy', 'settled': True, 'usd_volume': '285.1342000000000000'}, 'timestamp': 1609377350441, 'datetime': '2020-12-31T01:15:50.441Z', 'symbol': 'BTC/USD', 'type': None, 'takerOrMaker': 'taker', 'side': 'buy', 'price': 28513.42, 'amount': 0.01, 'fee': {'cost': 1.425671, 'currency': 'USD', 'rate': 0.005}, 'cost': 285.13419999999996}, {'id': '21715483', 'order': '573b5590-ca72-49a8-98af-b14e1dc28557', 'info': {'created_at': '2020-12-31T01:16:07.143Z', 'trade_id': 21715483, 'product_id': 'BTC-USD', 'order_id': '573b5590-ca72-49a8-98af-b14e1dc28557', 'user_id': '5fe53030706b8613d795bd7

In [55]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import sys
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers

class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

class TestStrategy(bt.Strategy):
    def next(self):
        for data in self.datas:
            print('*' * 5, 'NEXT:', bt.num2date(data.datetime[0]), data._name, data.open[0],
                  data.high[0], data.low[0], data.close[0], data.volume[0],
                  bt.TimeFrame.getname(data._timeframe), len(data))
            if self.broker.get_cash() > 100:
                self.buy(size=0.001)

    def notify_order(self, order):
        print('*' * 5, "NOTIFY ORDER", order)

cerebro = bt.Cerebro()

from_date=datetime.datetime(2016,1,1)
to_date = datetime.datetime(2020,1,1)

# Create data feeds
data_ticks = bt.feeds.CCXT(exchange='coinbasepro', symbol='BTC/USD', name="btc_usd_tick",
                           timeframe=bt.TimeFrame.Days, compression=1, fromdate=from_date, todate=to_date)
cerebro.adddata(data_ticks)

cerebro.broker.setcash(10000.0)
cerebro.broker.addcommissioninfo(CommInfoFractional())
cerebro.broker.setcommission(commission=0.005, margin=False, mult=1.0, commtype=None, percabs=True, stocklike=False, interest=0.0, interest_long=False, leverage=1.0, automargin=False, name=None)

cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='tradeanalyzer')


# Add the strategy
cerebro.addstrategy(TestStrategy)

# Run the strategy
strats = cerebro.run()

***** NEXT: 2016-01-01 00:00:00 btc_usd_tick 430.35 437.15 427.92 435.66 3863.27745054 Day 1
***** NOTIFY ORDER Ref: 4057
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 735964.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4057
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 735964.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4057
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 735964.9999999999
Info: AutoOrderedDict()
Broker: None
Aliv

***** NOTIFY ORDER Ref: 4102
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736009.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4102
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736009.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4102
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 736009.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2016-02-16 00:00:00 btc_usd_tick 398.94 407.5 390.0 407.42 5649.43557069

***** NOTIFY ORDER Ref: 4133
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736040.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4133
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736040.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4133
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 736040.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2016-03-18 00:00:00 btc_usd_tick 418.42 418.66 399.21 409.65 8597.586763

***** NOTIFY ORDER Ref: 4172
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736079.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4172
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736079.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4172
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 736079.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2016-04-26 00:00:00 btc_usd_tick 466.17 472.11 464.01 470.59 7471.240561

***** NOTIFY ORDER Ref: 4214
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736121.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4214
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 736121.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 4214
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 736121.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2016-06-07 00:00:00 btc_usd_tick 586.75 590.95 475.0 579.71 10079.836841

...

***** NOTIFY ORDER Ref: 5334
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737241.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5334
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737241.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5334
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 737241.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2019-07-02 00:00:00 btc_usd_tick 10578.23 10925.0 9651.0 10829.18 43614.

***** NOTIFY ORDER Ref: 5375
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737282.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5375
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737282.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5375
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 737282.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2019-08-12 00:00:00 btc_usd_tick 11540.76 11555.55 11222.4 11389.28 6095

***** NOTIFY ORDER Ref: 5417
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737324.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5417
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737324.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5417
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 737324.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2019-09-23 00:00:00 btc_usd_tick 10026.79 10051.15 9609.37 9693.74 11355

***** NOTIFY ORDER Ref: 5452
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737359.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5452
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737359.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5452
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 737359.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2019-10-28 00:00:00 btc_usd_tick 9555.89 9939.69 9180.59 9226.5 16736.97

***** NOTIFY ORDER Ref: 5494
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737401.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5494
OrdType: 0
OrdType: Buy
Status: 2
Status: Accepted
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737401.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 5494
OrdType: 0
OrdType: Buy
Status: 4
Status: Completed
Size: 0.001
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: <backtrader.comminfo.CommInfoBase object at 0x7fad4f5b7908>
End of Session: 737401.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2019-12-09 00:00:00 btc_usd_tick 7520.0 7659.38 7262.87 7337.51 10252.89

In [62]:
print('Return:', strats[0].analyzers.returns.get_analysis())
print('Sharpe Ratio:', strats[0].analyzers.mysharpe.get_analysis())
print('AR Ratio:', strats[0].analyzers.annualreturn.get_analysis())
print('Drawdown:', strats[0].analyzers.drawdown.get_analysis())
print('Drawdown:', strats[0].analyzers.tradeanalyzer.get_analysis())
cerebro.plot()

AttributeError: 'ItemCollection' object has no attribute 'returns'